In [1]:
from bs4 import BeautifulSoup
import urllib
import urllib.error
import urllib.request
import json
import pandas as pd
import numpy as np


URL_HOME = 'https://www.winamax.fr/paris-sportifs'
URL_MATCH = 'https://www.winamax.fr/paris-sportifs/match/'
URL_SPORT = 'https://www.winamax.fr/paris-sportifs/sports/'
URL_BOOST = 'https://www.winamax.fr/paris-sportifs/sports/100000'


def request_get_html(url):
    req = urllib.request.Request(
        url, 
        headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    })
    page = urllib.request.urlopen(req, timeout=10).read()
    parser = BeautifulSoup(page, 'html.parser')
    return parser

def winamax_get_dict_data(html):
    for line in html.find_all(['script']):
        if "PRELOADED_STATE" in str(line.string):
            txt_line = line.string
            txt_line = txt_line.split('var PRELOADED_STATE = ')[1][:-1]
            d_line = json.loads(txt_line)
    return d_line


## Retrieve list sports ID
req = request_get_html(URL_HOME)
d_data = winamax_get_dict_data(req)
d_mapping_sport = {}
for k in d_data['sports']:
    d_mapping_sport[k] = {}
    d_mapping_sport[k]['sportId'] = k
    d_mapping_sport[k]['sportName'] = d_data['sports'][str(k)]['sportName']
    
df_sports = pd.DataFrame.from_dict(d_mapping_sport, orient='index')
l_sport_ids = df_sports['sportId'].values.tolist()
l_sport_ids.remove('100000')
arr_sport_ids = np.array(l_sport_ids)

In [58]:
l_match_cols = ['matchId', 'available', 'status', 'title', 'sportId',
                'categoryId', 'tournamentId', 'competitor1Id',
                'competitor1Name', 'competitor2Id', 'competitor2Name']

l_bet_cols = ['betId', 'matchId', 'marketId', 'outcomes', 'available', 
              'betTypeIsLive', 'betTitle','betTypeName', 'betType', 
              'betTypeCategoryId', 'betTypeCategory', 'betGroup']

l_odd_cols = ['outcomeId', 'betId', 'matchId', 'label', 
              'available', 'code', 'percentDistribution', 'odd']

In [3]:
l_matches = []
l_bets = []
l_odds = []

for sport_id in arr_sport_ids:
    url = URL_SPORT + str(5)
    req = request_get_html(url)
    d_data = winamax_get_dict_data(req)
    for match_id in d_data['matches']:
        row = d_data['matches'][match_id]
        match_row = { key: row[key] for key in l_match_cols }
        l_matches.append(match_row)
        
df_matches = pd.DataFrame.from_records(l_matches)


for match_id in df_matches['matchId'].values:
    url = URL_MATCH + str(match_id)
    req = request_get_html(url)
    d_data = winamax_get_dict_data(req)
    for bet_id in d_data['bets']:
        row = d_data['bets'][bet_id]
        row['matchId'] = match_id
        bet_row = { key: row[key] for key in l_bet_cols }
        l_bets.append(bet_row)
        
        for odd in row['outcomes']:
            d_odd_row = d_data['outcomes'][str(odd)]
            d_odd_row['matchId'] = match_id
            d_odd_row['betId'] = bet_id
            d_odd_row['outcomeId'] = odd
            d_odd_row['odd'] = d_data['odds'][str(odd)]
            bet_odd = { key: d_odd_row[key] for key in l_odd_cols }
            l_odds.append(bet_odd)
        break
    break
    
df_bets = pd.DataFrame.from_records(l_bets)
df_odds = pd.DataFrame.from_records(l_odds)